# Gaussian 中的 PUHF/PMP2 结果的重新实现

在这一份笔记中，我们将使用 PySCF 的功能与 NumPy 重复 Gaussian 中计算的 PUHF 与 PMP2 能量结果。

<div class="alert alert-warning">

**请求**

尽管这份文档应当成功地重复了 Gaussian 所给出的 PUHF 与 PMP2 能量，但作者目前并不知道这两个能量结果的理论基础。若读者对理论背景有所了解，欢迎前往 [文档的 github](https://github.com/ajz34/ajz34.readthedocs.io/issues) 提 issue。

</div>

In [1]:
from pyscf import gto, scf, mp

## 结果与体系定义

### Gaussian 结果

在 Gaussian 中，我们使用以下输入卡可以得到 PUHF/PMP2 能量：

```
#p UMP2(Full)/6-31G nosymm

H2O

1 2
O 0. 0. 0.
H 1. 0. 0.
H 0. 1. 0.
```

对于上述分子，其中一些重要的输出结果是

1. $E_\mathrm{UHF}$：-75.5663698168

2. $E_\mathrm{UMP2, corr}$: -0.09541598704

3. $E_\mathrm{UMP2}$: -75.661785803869

4. $\langle S_z \rangle$: 0.5

5. $\langle S^{2(0)} \rangle$: 0.75677

6. $\langle S^{2(0)} \rangle + \langle S^{2(1)} \rangle$: 0.75288

7. $E_\mathrm{PUHF}$：-75.568214846

8. $E_\mathrm{PMP2}$: -75.663102325

输出文件参见 [PUHF_and_PMP2.out](assets/PUHF_and_PMP2.out)；其中，有效的数据可以通过下述的代码获得：

In [2]:
with open("assets/PUHF_and_PMP2.out", "r") as output:
    output_lines = output.read().split("\n")

for line_num, line_text in enumerate(output_lines):
    if any([keyword in line_text for keyword in
            ["SCF Done", "EUMP2", "<S**2>", "(S**2,1)", "E(PMP2)"]]) \
        and "Initial guess" not in line_text:
        print("line {:03d}: {}".format(line_num, line_text))

line 349:  SCF Done:  E(UHF) =  -75.5663698168     A.U. after   12 cycles
line 351:  <Sx>= 0.0000 <Sy>= 0.0000 <Sz>= 0.5000 <S**2>= 0.7568 S= 0.5034
line 383:  (S**2,0)=  0.75677D+00           (S**2,1)=  0.75288D+00
line 384:  E(PUHF)=      -0.75568214846D+02        E(PMP2)=      -0.75663102325D+02
line 386:  E2 =    -0.9541598704D-01 EUMP2 =    -0.75661785803869D+02


我们的目标就是近乎准确无误地重复上述八个结果。

### PySCF 体系定义

为了获得与 Gaussian 相同的结果，我们需要定义相同的分子与电荷、多重度环境：

In [3]:
mol = gto.Mole()
mol.atom = """
O 0. 0. 0.
H 1. 0. 0.
H 0. 1. 0.
"""
mol.charge = 1
mol.spin = 1
mol.basis = "6-31G"
mol.build()

通过 PySCF 计算 UHF 能量：

In [4]:
scf_eng = scf.UHF(mol)
scf_eng.run();

converged SCF energy = -75.5663698072169  <S^2> = 0.75677134  2S+1 = 2.0067599


上述结果应当能与 $E_\mathrm{UHF}$ 和 $\langle S^{2(0)} \rangle$ 对应。$\langle S_z \rangle = 0.5$ 几乎是显然的。不过，我们仍然不了解 $\langle S^{2(0)} \rangle$ 是如何生成的。

通过 PySCF 计算 UMP2 能量：

In [5]:
mp2_eng = mp.UMP2(scf_eng)
mp2_eng.run();

E(UMP2) = -75.6617858050029  E_corr = -0.0954159977860057


上述结果应当能与 $E_\mathrm{UMP2, corr}$ 和 $E_\mathrm{UMP2}$ 对应。

因此，当前的问题将是回答：如何重复

1. $\langle S^{2(0)} \rangle$: 0.75677

2. $\langle S^{2(0)} \rangle + \langle S^{2(1)} \rangle$: 0.75288

3. $E_\mathrm{PUHF}$：-75.568214846

4. $E_\mathrm{PMP2}$: -75.663102325

### 部分变量定义

首先，我们遵从大多数量化文章中的记号

- $i, j$ 代表占据分子轨道

- $a, b$ 代表非占分子轨道

- $p, q$ 代表任意分子轨道

- $\alpha, \beta$ 代表任意原子轨道

<center><b>Table 1. 分子相关变量</b></center>

| 变量名 | 元素记号 | 意义与注解 | 标量或区间 |
|-|-|-|-|
| `nocc_a` | $n_\mathrm{occ}^\alpha$ | $\alpha$ 自旋电子数 | $5$ |
| `nocc_b` | $n_\mathrm{occ}^\beta$ | $\beta$ 自旋电子数 | $4$ |
| `nmo` | $n_\mathrm{MO}$ | 分子轨道数 | $13$ |
| `nao` | $n_\mathrm{AO}$ | 原子轨道数 | $13$ |
| `S` | $S_{\mu \nu}$ | 重叠积分 | - |
| `so_a` | - | $\alpha$ 占据轨道分割 | $[0, 5)$ |
| `so_b` | - | $\beta$ 占据轨道分割 | $[0, 4)$ |
| `sv_a` | - | $\alpha$ 非占轨道分割 | $[5, 13)$ |
| `sv_b` | - | $\beta$ 非占轨道分割 | $[4, 13)$ |
| `Sx` | $S_x$ | $x$ 分量自旋 | $0$ |
| `Sy` | $S_y$ | $y$ 分量自旋 | $0$ |
| `Sz` | $S_z$ | $z$ 分量自旋 | $1/2$ |

<center><b>Table 2. UHF 计算相关变量</b></center>

| 变量名 | 元素记号 | 意义与注解 |
|-|-|-|
| `C_a` | $C_{\mu p}^\alpha$ | $\alpha$ 系数矩阵 |
| `C_b` | $C_{\mu p}^\beta$ | $\beta$ 系数矩阵 |
| `D_a` | $C_{\mu \nu}^\alpha$ | $\alpha$ 密度矩阵 |
| `D_b` | $D_{\mu \nu}^\beta$ | $\beta$ 密度矩阵 |
| `e_a` | $e_{p}^\alpha$ | $\alpha$ 轨道能 |
| `e_b` | $e_{p}^\beta$ | $\alpha$ 轨道能 |
| `eo_a` | $e_{i}^\alpha$ | $\beta$ 占据轨道能 |
| `eo_b` | $e_{i}^\beta$ | $\alpha$ 占据轨道能 |
| `ev_a` | $e_{a}^\alpha$ | $\alpha$ 非占轨道能 |
| `ev_b` | $e_{a}^\beta$ | $\beta$ 非占轨道能 |
| `D2_aa` | $D_{ij}^{ab, \alpha \alpha}$ | $\alpha, \alpha$ 轨道能差 |
| `D2_ab` | $D_{ij}^{ab, \alpha \beta}$ | $\alpha, \beta$ 轨道能差 |
| `D2_bb` | $D_{ij}^{ab, \beta \beta}$ | $\beta, \beta$ 轨道能差 |

<center><b>Table 3. UMP2 计算相关变量</b></center>

| 变量名 | 元素记号 | 意义与注解 |
|-|-|-|
| `t2_aa` | $t_{ij}^{ab, \alpha \alpha}$ | $\alpha, \alpha$ MP2 激发系数 |
| `t2_ab` | $t_{ij}^{ab, \alpha \beta}$ | $\alpha, \beta$ MP2 激发系数 |
| `t2_bb` | $t_{ij}^{ab, \beta \beta}$ | $\beta, \beta$ MP2 激发系数 |
| `D2_aa` | $D_{ij}^{ab, \alpha \alpha}$ | $\alpha, \alpha$ MP2 激发系数分母 |
| `D2_ab` | $D_{ij}^{ab, \alpha \beta}$ | $\alpha, \beta$ MP2 激发系数分母 |
| `D2_bb` | $D_{ij}^{ab, \beta \beta}$ | $\beta, \beta$ MP2 激发系数分母 |

上述需要补充说明的公式有：

$$
S_z = \frac{1}{2} (n_\mathrm{occ}^\alpha - n_\mathrm{occ}^\beta)
$$

$$
D_{ij}^{ab, \sigma \sigma'} = e_i^{\sigma} + e_j^{\sigma'} - e_a^{\sigma} - e_b^{\sigma'}
$$

需要指出：

- 程序中四角标 $D_{ij}^{ab, \sigma \sigma'}$ 或 $t_{ij}^{ab, \sigma \sigma'}$ 张量的索引顺序是 $i, j, a, b$；这与我以前的 [pyxdh 项目](https://github.com/ajz34/Py_xDH) 的顺序 $i, a, j, b$ 不同。
- 轨道记号 $i, j, a, b, p, q$ 在这里应当是自旋轨道，这里出于简化公式表示的目的，就没有在其旁边补上自选方向；取而代之地，矩阵或张量的右上角会说明轨道的自旋。
    - 对于类似 $R_{p}^\sigma$ 或 $R_{\mu p}^\sigma$ 的记号，只有一个分子轨道 $p$，因此 $p$ 指代 $\sigma$ 自旋；
    - 对于类似 $R_{pq}^{\sigma \sigma'}$ 或 $R_{pq}^{rs, \sigma \sigma'}$ 的记号，左侧的 $p, r$ 指代 $\sigma$ 自旋，而右侧的 $q, s$ 指代 $\sigma'$ 自旋。

In [6]:
# === Molecular
# --- Definition
nocc_a, nocc_b = mol.nelec
nmo = nao = mol.nao
S = mol.intor("int1e_ovlp")
# --- Derivative
so_a, so_b = slice(0, nocc_a), slice(0, nocc_b)
sv_a, sv_b = slice(nocc_a, nmo), slice(nocc_b, nmo)
Sx, Sy, Sz = 0, 0, 0.5 * (nocc_a - nocc_b)

# === UHF Calculation
# --- Definition
C_a, C_b = scf_eng.mo_coeff
D_a, D_b = scf_eng.make_rdm1()
e_a, e_b = scf_eng.mo_energy
# --- Derivative
eo_a, eo_b = e_a[so_a], e_b[so_b]
ev_a, ev_b = e_a[sv_a], e_b[sv_b]
D2_aa = eo_a[:, None, None, None] + eo_a[None, :, None, None] - ev_a[None, None, :, None] - ev_a[None, None, None, :]
D2_ab = eo_a[:, None, None, None] + eo_b[None, :, None, None] - ev_a[None, None, :, None] - ev_b[None, None, None, :]
D2_bb = eo_b[:, None, None, None] + eo_b[None, :, None, None] - ev_b[None, None, :, None] - ev_b[None, None, None, :]

# === MP2 Calculation
t2_aa, t2_ab, t2_bb = mp2_eng.t2

作为对四脚标张量性质的验证，我们计算 MP2 相关能 $E_\mathrm{MP2, corr}$ 如下：

$$
E_\mathrm{MP2, corr} =
\frac{1}{4} \sum_{i^\alpha j^\alpha a^\alpha b^\alpha} (t_{ij}^{ab, \alpha \alpha})^2 D_{ij}^{ab, \alpha \alpha} +
\frac{1}{4} \sum_{i^\beta j^\beta a^\beta b^\beta} (t_{ij}^{ab, \beta \beta})^2 D_{ij}^{ab, \beta \beta} +
\sum_{i^\alpha j^\beta a^\alpha b^\beta} (t_{ij}^{ab, \alpha \beta})^2 D_{ij}^{ab, \alpha \beta}
$$

In [7]:
(+ 0.25 * (t2_aa**2 * D2_aa).sum()
 + 0.25 * (t2_bb**2 * D2_bb).sum()
 + (t2_ab**2 * D2_ab).sum())

-0.09541599778600572

而 PySCF 所给出的 $E_\mathrm{MP2, corr}$：

In [8]:
mp2_eng.e_corr

-0.09541599778600571

## UHF 相关计算

### `S_pq` $S_{pq}^{\alpha \beta}$

$$
S_{pq}^{\alpha \beta} = \sum_{\mu \nu} C_{\mu p}^\alpha S_{\mu \nu} C_{\nu q}^\beta
$$

若用量子力学记号，上述矩阵元素可能表示为

$$
S_{pq}^{\alpha \beta} = \langle p^\alpha | q^\beta \rangle
$$

In [9]:
S_pq = C_a.T @ S @ C_b
S_pq.shape

(13, 13)

我们以后还会使用上述矩阵的占据-占据部分 `S_ij` $S_{ij}^{\alpha \beta}$：

In [10]:
S_ij = S_pq[so_a, so_b]
S_ij.shape

(5, 4)

### `S2_0` $\langle S^{2(0)} \rangle$

$\langle S^{2(0)} \rangle$ 在程序中通常写为 `<S^2>` 或 `<S**2>`。在 Gaussian 计算 PUHF 处，还写为 `(S**2,0)`。这意味着是 UHF 波函数的 $\langle S^2 \rangle_\mathrm{UHF}$。相对地，UMP2 波函数给出的对 $\langle S^2 \rangle$ 的矫正将记作 $\langle S^{2(1)} \rangle$。

$$
\langle S^{2(0)} \rangle = \frac{1}{2} (n_\mathrm{occ}^\alpha + n_\mathrm{occ}^\beta) + S_x^2 + S_y^2 + S_z^2 - \sum_{i^\alpha j^\beta} (S_{ij}^{\alpha \beta})^2
$$

In [11]:
S2_0 = 0.5 * (nocc_a + nocc_b) + Sx**2 + Sy**2 + Sz**2 - (S_ij**2).sum()
S2_0

0.7567713384723822

### `S4SD` $\texttt{S4SD}$

`S4SD` 的表达式较为复杂，我们也直接使用 $\texttt{S4SD}$ 而不用其他记号表示该项：

$$
\begin{align}
\texttt{S4SD} =
&(S_z (S_z + 1) + n_\mathrm{occ}^\beta)^2 + n_\mathrm{occ}^\alpha n_\mathrm{occ}^\beta + 2 \, \sum_{i^\alpha j^\beta} \big( S_{ij}^{\alpha \beta} \big)^2 \\
&+ \big( 2 S_z^2 + 2 (n_\mathrm{occ}^\alpha + n_\mathrm{occ}^\beta) - 2 \big) \, \sum_{i^\alpha j^\beta} \big( S_{ij}^{\alpha \beta} \big)^2 \\
&- 2 \sum_{i^\alpha j^\beta k^\alpha l^\beta} S_{ij}^{\alpha \beta} S_{kj}^{\alpha \beta} S_{kl}^{\alpha \beta} S_{il}^{\alpha \beta}
\end{align}
$$

由于上述公式比较复杂，我们按如下方式定义分项：

$$
\begin{align}
\texttt{RD} &= S_z (S_z + 1) + n_\mathrm{occ}^\beta \\
\texttt{TT2} &= 2 S_z^2 + 2 (n_\mathrm{occ}^\alpha + n_\mathrm{occ}^\beta) - 2 \\
\texttt{TrPQ1} &= \sum_{i^\alpha j^\beta} \big( S_{ij}^{\alpha \beta} \big)^2 = \sum_{i^\alpha j^\beta} \langle i^\alpha | j^\beta \rangle \langle j^\beta | i^\alpha \rangle \\
\texttt{TrPQ2} &= \sum_{i^\alpha j^\beta k^\alpha l^\beta} S_{ij}^{\alpha \beta} S_{kj}^{\alpha \beta} S_{kl}^{\alpha \beta} S_{il}^{\alpha \beta} \\
&= \sum_{i^\alpha j^\beta k^\alpha l^\beta} \langle i^\alpha | j^\beta \rangle \langle j^\beta | k^\alpha \rangle \langle k^\alpha | l^\beta \rangle \langle l^\beta | i^\alpha \rangle
\end{align}
$$

那么

$$
\texttt{S4SD} = \texttt{RD}^2 + n_\mathrm{occ}^\alpha n_\mathrm{occ}^\beta + 2 \times \texttt{TrPQ1}^2 + \texttt{TT2} \times \texttt{TrPQ1} - 2 \times \texttt{TrPQ2}
$$

In [12]:
RD = Sz * (Sz + 1) + nocc_b
TT2 = 2 * Sz**2 + 2 * (nocc_a + nocc_b) - 2
TrPQ1 = (S_ij**2).sum()
TrPQ2 = (S_ij @ S_ij.T @ S_ij @ S_ij.T).trace()

In [13]:
S4SD = RD**2 + nocc_a * nocc_b + 2 * TrPQ1**2 - 2 * TrPQ2 - TT2 * TrPQ1
S4SD

0.5930233835502321

## UMP2 相关计算

### `S2_1` $\langle S^{2(1)} \rangle$

$$
\langle S^{2(1)} \rangle = - 2 \sum_{i^\alpha j^\beta a^\alpha b^\beta} t_{ij}^{ab, \alpha \beta} S_{ib}^{\alpha \beta} S_{aj}^{\alpha \beta} = - 2 \sum_{i^\alpha j^\beta a^\alpha b^\beta} t_{ij}^{ab, \alpha \beta} \langle i^\alpha | b^\beta \rangle \langle a^\alpha | j^\beta \rangle
$$

In [14]:
S2_1 = - 2 * (t2_ab * S_pq[so_a, None, None, sv_b] * S_pq.T[None, so_b, sv_a, None]).sum()
S2_1

-0.0038892907027640506

因此，UMP2 矫正过的 $\langle S^2 \rangle_\mathrm{UMP2} = \langle S^{2(0)} \rangle + \langle S^{2(1)} \rangle$ 的结果是

In [15]:
S2_0 + S2_1

0.7528820477696182

Gaussian 的参考值是 0.75288。

### `PUHF` $\texttt{PUHF}$

$$
\texttt{PUHF} = - \sum_{i^\alpha j^\beta a^\alpha b^\beta} t_{ij}^{ab, \alpha \beta} D_{ij}^{ab, \alpha \beta} S_{ib}^{\alpha \beta} S_{aj}^{\alpha \beta}
= - \sum_{i^\alpha j^\beta a^\alpha b^\beta} \langle i^\alpha j^\beta \Vert a^\alpha b^\beta \rangle \langle i^\alpha | b^\beta \rangle \langle a^\alpha | j^\beta \rangle
$$

In [16]:
PUHF = - (t2_ab * D2_ab * S_pq[so_a, None, None, sv_b] * S_pq.T[None, so_b, sv_a, None]).sum()

该 `PUHF` $\texttt{PUHF}$ 是一个中间变量，**并非**经过自旋污染矫正的能量 $E_\mathrm{PUHF}$。

## 自旋污染矫正能量计算

### `EPUHF` $E_\mathrm{PUHF}$

为了程序方便，定义下述临时变量

$$
\texttt{Y} = \langle S^{2(0)} \rangle - (S_z + 1) (S_z + 2)
$$

那么

$$
E_\mathrm{PUHF} = E_\mathrm{UHF} + \frac{\texttt{PUHF}}{\texttt{Y}}
= E_\mathrm{UHF} - \frac{\sum_{i^\alpha j^\beta a^\alpha b^\beta} \langle i^\alpha j^\beta \Vert a^\alpha b^\beta \rangle \langle i^\alpha | b^\beta \rangle \langle a^\alpha | j^\beta \rangle}{\langle S^{2(0)} \rangle - (S_z + 1) (S_z + 2)}
$$

In [17]:
Y = S2_0 - (Sz + 1) * (Sz + 2)
EPUHF = scf_eng.e_tot + PUHF / Y
EPUHF

-75.56821484772901

Gaussian 的参考值是 -75.568214846。

### `EPMP2` $E_\mathrm{PMP2}$

$$
\begin{align}
E_\mathrm{PMP2} &= E_\mathrm{PUHF} + E_\mathrm{MP2, corr} - \frac{\texttt{PUHF}}{2} \frac{\langle S^{2(1)} \rangle}{\texttt{S4SD} - \langle S^{2(0)} \rangle^2} \\
&= E_\mathrm{PUHF} + E_\mathrm{MP2, corr} + \frac{1}{2} \frac{\langle S^{2(1)} \rangle}{\texttt{S4SD} - \langle S^{2(0)} \rangle^2} \sum_{i^\alpha j^\beta a^\alpha b^\beta} \langle i^\alpha j^\beta \Vert a^\alpha b^\beta \rangle \langle i^\alpha | b^\beta \rangle \langle a^\alpha | j^\beta \rangle
\end{align}
$$

In [18]:
EPMP2 = EPUHF + mp2_eng.e_corr - 0.5 * PUHF * S2_1 / (S4SD - S2_0**2)
EPMP2

-75.66310233784893

Gaussian 的参考值是 -75.663102325。

至此，我们已经完成了使用 PySCF 的功能与 NumPy 重复 Gaussian 的 PUHF、PMP2 的能量结果了。